In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from random import shuffle
from music21 import *
from glob import glob
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, LSTM, RepeatVector, Dropout
from keras.utils import np_utils

%run lstm_model.ipynb
np.set_printoptions(threshold=1000000)
bach_chorale_path = 'datasets/bach2'
parts_index = [0,1,2,3]
#parts_index = [0]
note_type = set()
"""
note_type:
0-127 : note pitch
128: slur-1st-part
129: slur-2nd-part
130: slur-3rd-part
131: slur-4th-part
132: rest
133: start symbol
134: end symbol
"""

def filter_files(files):
    file_list = []
    file_count = 0
    for i, file in enumerate(files, 1):
        s = converter.parse(file)
        if len(s.parts) == len(parts_index):
            print(i, file, len(s.parts))
            file_count += 1
            file_list.append(file)
    return file_list, file_count

def chorale_to_inputs(s,i):
    inputs = []
    for j in range(16):
        inputs.append([133])
    for n in s.parts[i].flat.notes:
#         if n.isNote:
#             print(n.nameWithOctave, int(4 * n.quarterLength), n.offset)
#         else:
#             print(n.pitchNames, int(4 * n.quarterLength), n.offset)
        #print(n.expressions)
        if n.isNote:
            inputs.append([n.pitch.midi])
            note_type.add(frozenset([n.pitch.midi]))
        elif n.isChord:
            inputs.append([_note.pitch.midi for _note in n])
            note_type.add(frozenset([_note.pitch.midi for _note in n]))
        l = n.quarterLength
#         if l == 0:
#             l = 1
        for j in range(1, int(4 * l)):
            if i == 0:
                inputs.append([128])
                note_type.add(frozenset([128]))
            elif i == 1:
                inputs.append([129])
                note_type.add(frozenset([129]))
            elif i == 2:
                inputs.append([130])
                note_type.add(frozenset([130]))
            elif i == 3:
                inputs.append([131])
                note_type.add(frozenset([131]))
    for j in range(16):
        inputs.append([134])
    return inputs

def make_time_metadata(file):
    chorale = converter.parse(file)
    chorale_length = int(chorale.parts[0].duration.quarterLength * 4)
    time_metadata = np.zeros((chorale_length, ))
    
    for i in range(0, chorale_length):
        time_metadata[i] = i + 1
    return np.array(time_metadata, dtype=np.int32)

def append_notes(max_length, part_i_length, part_i_input):
    note_type.add(frozenset([132]))# 132-->note.Rest
    for num_appended_notes in range(max_length - part_i_length):
        part_i_input.append([132])
    return part_i_input

def check_parts_length(parts_length, inputs):
    max_length = max(parts_length)
    for i in parts_index:
        if parts_length[i] < max_length:
            inputs[i] = append_notes(max_length, parts_length[i], inputs[i])
                
def make_dataset(file_list):
    X = []
    time_metadata = []
    for file in tqdm(file_list):
        s = converter.parse(file)
        
        inputs = []
        parts_length = []
        time_metadata.append(make_time_metadata(file))
        for i in parts_index:
            part_input = chorale_to_inputs(s,i)
            parts_length.append(len(part_input))
            inputs.append(part_input)
        check_parts_length(parts_length, inputs)
        np_inputs = np.array(inputs)
        X.append(np_inputs)
    #X.reshape(X.shape[1:])
    #print(time_metadata)
    return X
def generator_X(notes):
    t = 0
    while True:
        X = []
        if t < notes.shape[0] - 16:
            X.append(notes[t:t+16])
        else: 
            t = 0
            continue
        t += 16
        X = np.array(X)
        yield (X)
def generate_notes(lstm_model, X):
    output_indices = []
    for i in range(len(parts_index)):
        model = load_model("models/bach"+str(i)+".h5")
#     model = load_model(lstm_model.name+".h5")
        gen = ((X) for (X) in generator_X(lstm_model.X))
#     predict_list = model.predict(X, batch_size=32)
        predict_list = model.predict_generator(gen, steps=500)
        part_notes = []
        for time in range(predict_list.shape[0]):
            for note_index in range(predict_list.shape[1]):
                part_notes.append(np.argmax(predict_list[time][note_index]))
        output_indices.append(part_notes)
    output_indices = np.array(output_indices)
    output_chorale = index_note_transform(output_indices, "to_note", len(parts_index))
    #output_indices = np.transpose(output_indices)
   # output_chorale = np.transpose(output_chorale)
    midi = chorale_to_midi(output_chorale)
    print(output_chorale)
    return midi

# def insert_note_chord(n, d, new_note, part):
    
# def notes_to_midi(n, d, new_note, part):
#     if (len(n) < 2): # note or rest
#         if n[0] == 132:
#             part.append(note.Rest())
#         elif n[0] < 128:
#             if d > 0:
#                 new_note.duration = duration.Duration(d / 4)
#                 part.append(new_note)
#                 d = 1
#                 new_note = note.Note(n[0])
#             else:
#                 d += 1
#     else: # chord
#         if d > 0:
#             new_note.duration = duration.Duration(d / 4)
#             part.append(new_note)
#             d = 1
#             new_note = chord.Chord(n)
#         else:
#             d += 1
def chorale_to_midi(chorale):
    score = stream.Score()
    for i, chorale_part in enumerate(chorale):
        part = stream.Part(id=i)
        d = 0
        new_note = note.Rest()
        for sound in chorale_part:
            sound = [n for n in sound]
            if (len(sound) < 2): # note or rest
                if sound[0] == 132:
                    part.append(note.Rest())
                elif sound[0] < 128:
                    if d > 0:
                        new_note.duration = duration.Duration(d / 4)
                        part.append(new_note)
                    d = 1
                    new_note = note.Note(sound[0])
                else:
                    d += 1
            else: # chord
                if d > 0:
                    new_note.duration = duration.Duration(d / 4)
                    part.append(new_note)
                d = 1
                new_note = chord.Chord(sound)
            
        new_note.duration = duration.Duration(d / 4)
        part.append(new_note)
        score.insert(part)
    return score
def make_note_type_list():
    note_type_list = []
    for i in note_type:
        note_type_list.append(i)
    return sorted(note_type_list)
# def transform_note_to_index(notes):
#     for time in range(notes.shape[0]):
#         for p_index in range(4):
#             notes[time][p_index] = note_dict[notes[time][p_index]]
#     return notes
def index_note_transform(inputs, option, pn):
    transformed_inputs = []
    if option == "to_note":
        note_dict = dict((index, note) for index, note in enumerate(note_type))
    elif option == "to_index":
        note_dict = dict((note, index) for index, note in enumerate(note_type))
        
    for p_index in range(pn):
        transformed_part = []
        for n in range(inputs.shape[1]):
            for keys in note_dict:
                if option == "to_note":
                    if inputs[p_index][n] == keys:
                        transformed_part.append(note_dict[keys])
                elif option == "to_index":
                    if frozenset(inputs[p_index][n]) == keys:
                        transformed_part.append(note_dict[keys])
        transformed_inputs.append(transformed_part)
    return np.array(transformed_inputs)
def notes_to_piano_roll(dataset_notes):
#     start_symbol = np.full((16, 1),133)
#     end_symbol = np.full((16, 1),134)
    note_type.add(frozenset([133]))
    note_type.add(frozenset([134]))
    for chorale_index in range(len(dataset_notes)):
        chorale = np.array(dataset_notes[chorale_index])
#         for i in range(len(parts_index)):
#             piano_roll_notes.append(np.concatenate((start_symbol, chorale[i], end_symbol)))
        chorale = index_note_transform(chorale, "to_index", len(parts_index))
        piano_roll_notes = np.transpose(chorale)
        #piano_roll_notes = index_note_transform(transposed_chorale, "to_index", 4)
        yield piano_roll_notes
def notes_to_onehot(notes):
    note_dict = note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = []
        for note in n:
            a.append(np.array(note == np.arange(0, len(note_dict)), dtype=np.int32))
        onehot.append(a)
    return np.array(onehot)
def part_notes_to_onehot(notes):
    note_dict = dict((index, note) for index, note in enumerate(note_type))
    onehot = []
    for i, n in enumerate(notes):
        a = np.zeros(len(note_dict,),dtype=np.int32)
        for note in n:
            a[note] = 1
        onehot.append(a)
    return np.array(onehot)

def load_files():
    if os.path.exists(bach_chorale_path):
#         file_list = filter_files(glob(bach_chorale_path))
        file_list, num_file = filter_files(glob(bach_chorale_path + '/*.mxl') +
                                 glob(bach_chorale_path + '/*.mid'))
        print(num_file)
        dataset_notes = make_dataset(file_list)
        shuffle(dataset_notes)
        chorale_list = [notes for chorale_index, notes in enumerate(notes_to_piano_roll(dataset_notes))]
        X_train = chorale_list[0]
        X_test = chorale_list[int(len(chorale_list) * 0.8)]
        notes = chorale_list[0]
        
        for i, n in enumerate(chorale_list):
            if i != 0:
                notes = np.concatenate((notes, n))
                if i < int(len(chorale_list) * 0.8):
                    X_train = np.concatenate((X_train, n))
                elif i > int(len(chorale_list) * 0.8):
                    X_test = np.concatenate((X_test, n))
        print(notes.shape)
    
        X_train_onehot_notes = part_notes_to_onehot(X_train)
        X_test_onehot_notes = part_notes_to_onehot(X_test)
        Y_train_onehot_notes = notes_to_onehot(X_train)
        Y_test_onehot_notes = notes_to_onehot(X_test)
        for i in range(len(parts_index)):
            lstm_model = LSTM_Model(X=X_train_onehot_notes,Y=Y_train_onehot_notes,
                                    num_units=75, epochs=25, output_dim=len(note_type), name="bach"+str(i))
            gen = ((X,Y) for (X, Y) in lstm_model.generator(X_train_onehot_notes, Y_train_onehot_notes, i))
            test_gen = ((X,Y) for (X,Y) in lstm_model.generator(X_test_onehot_notes, Y_test_onehot_notes, i))
            lstm_model.create_model()
            lstm_model.train_model(gen, test_gen)

        score = generate_notes(lstm_model, notes)
                
        output_file = str(lstm_model.epochs) + '-epochs_' + str(num_file) + '-samples.mid'
        mf = midi.translate.music21ObjectToMidiFile(score)
        mf.open(output_file, 'wb')
        mf.write()
        mf.close()
        print("File " + output_file + " written")
    else:
        print("The file path is wrong!!")
    
def main():
    load_files()
if __name__ == "__main__":
    main()

Using TensorFlow backend.


1 datasets/bach2\bach_302.mid 4
2 datasets/bach2\bach_303.mid 4
3 datasets/bach2\bach_304.mid 4
4 datasets/bach2\bach_305.mid 4
5 datasets/bach2\bach_306.mid 4
6 datasets/bach2\bach_307.mid 4
7 datasets/bach2\bach_308.mid 4
8 datasets/bach2\bach_309.mid 4
9 datasets/bach2\bach_310.mid 4
10 datasets/bach2\bach_311.mid 4
10


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  9.75it/s]


(2640, 4)
create model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 16, 75)            36900     
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 75)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 16, 75)            45300     
_________________________________________________________________
dropout_2 (Dropout)          (None, 16, 75)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 16, 47)            3572      
Total params: 85,772
Trainable params: 85,772
Non-trainable params: 0
_________________________________________________________________
train model...
Epoch 1/25
256/256 [==============================] - ETA: 414s - loss: 3.864 - ETA: 142s - loss:

256/256 [==============================] - ETA: 7s - loss: 0.637 - ETA: 8s - loss: 0.717 - ETA: 9s - loss: 0.721 - ETA: 9s - loss: 1.032 - ETA: 9s - loss: 1.157 - ETA: 10s - loss: 1.14 - ETA: 10s - loss: 1.16 - ETA: 10s - loss: 1.13 - ETA: 10s - loss: 1.12 - ETA: 11s - loss: 1.10 - ETA: 11s - loss: 1.15 - ETA: 11s - loss: 1.15 - ETA: 11s - loss: 1.15 - ETA: 11s - loss: 1.15 - ETA: 11s - loss: 1.15 - ETA: 11s - loss: 1.15 - ETA: 11s - loss: 1.14 - ETA: 10s - loss: 1.14 - ETA: 10s - loss: 1.13 - ETA: 10s - loss: 1.12 - ETA: 10s - loss: 1.12 - ETA: 10s - loss: 1.11 - ETA: 10s - loss: 1.10 - ETA: 10s - loss: 1.09 - ETA: 10s - loss: 1.09 - ETA: 10s - loss: 1.08 - ETA: 9s - loss: 1.1108 - ETA: 9s - loss: 1.101 - ETA: 10s - loss: 1.09 - ETA: 9s - loss: 1.0807 - ETA: 9s - loss: 1.073 - ETA: 9s - loss: 1.061 - ETA: 9s - loss: 1.058 - ETA: 9s - loss: 1.052 - ETA: 9s - loss: 1.040 - ETA: 9s - loss: 1.032 - ETA: 9s - loss: 1.030 - ETA: 9s - loss: 1.026 - ETA: 9s - loss: 1.024 - ETA: 9s - loss: 1.0

...

256/256 [==============================] - ETA: 7s - loss: 0.007 - ETA: 7s - loss: 0.019 - ETA: 7s - loss: 0.014 - ETA: 7s - loss: 0.014 - ETA: 7s - loss: 0.017 - ETA: 6s - loss: 0.015 - ETA: 6s - loss: 0.016 - ETA: 6s - loss: 0.016 - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.013 - ETA: 6s - loss: 0.015 - ETA: 6s - loss: 0.015 - ETA: 6s - loss: 0.015 - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.021 - ETA: 6s - loss: 0.020 - ETA: 6s - loss: 0.020 - ETA: 6s - loss: 0.019 - ETA: 6s - loss: 0.019 - ETA: 6s - loss: 0.019 - ETA: 6s - loss: 0.020 - ETA: 6s - loss: 0.019 - ETA: 6s - loss: 0.022 - ETA: 6s - loss: 0.021 - ETA: 6s - loss: 0.023 - ETA: 5s - loss: 0.022 - ETA: 5s - loss: 0.025 - ETA: 5s - loss: 0.025 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.024 - ETA: 5s - loss: 0.023 - ETA: 5s - loss: 0.023 - ETA: 5s - loss: 0.022 - ETA: 5s - loss: 0.022 - ETA: 5s - loss: 0.023 - ETA: 5s - loss: 0.023 - ETA: 5s - loss: 0.022

256/256 [==============================] - ETA: 6s - loss: 0.014 - ETA: 6s - loss: 0.005 - ETA: 6s - loss: 0.008 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.008 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.008 - ETA: 6s - loss: 0.008 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.009 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.009 - ETA: 6s - loss: 0.009 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.009 - ETA: 5s - loss: 0.009 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011

256/256 [==============================] - ETA: 7s - loss: 0.039 - ETA: 7s - loss: 0.017 - ETA: 7s - loss: 0.015 - ETA: 7s - loss: 0.011 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.012 - ETA: 7s - loss: 0.012 - ETA: 7s - loss: 0.011 - ETA: 7s - loss: 0.011 - ETA: 7s - loss: 0.010 - ETA: 7s - loss: 0.011 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.012 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.011 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 6s - loss: 0.010 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.011 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.010 - ETA: 5s - loss: 0.011

256/256 [==============================] - ETA: 6s - loss: 0.002 - ETA: 6s - loss: 0.003 - ETA: 6s - loss: 0.003 - ETA: 6s - loss: 0.005 - ETA: 7s - loss: 0.004 - ETA: 7s - loss: 0.008 - ETA: 7s - loss: 0.007 - ETA: 7s - loss: 0.008 - ETA: 7s - loss: 0.007 - ETA: 7s - loss: 0.009 - ETA: 7s - loss: 0.008 - ETA: 7s - loss: 0.008 - ETA: 6s - loss: 0.008 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.008 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.007 - ETA: 6s - loss: 0.006 - ETA: 6s - loss: 0.006 - ETA: 6s - loss: 0.006 - ETA: 6s - loss: 0.006 - ETA: 6s - loss: 0.006 - ETA: 6s - loss: 0.006 - ETA: 5s - loss: 0.006 - ETA: 5s - loss: 0.006 - ETA: 5s - loss: 0.006 - ETA: 5s - loss: 0.006 - ETA: 5s - loss: 0.006 - ETA: 5s - loss: 0.005 - ETA: 5s - loss: 0.005 - ETA: 5s - loss: 0.005 - ETA: 5s - loss: 0.005 - ETA: 5s - loss: 0.006

  frozenset({133}) frozenset({133}) frozenset({133}) frozenset({133})]]
File 25-epochs_10-samples.mid written


In [ ]:
n = [0,1,2,3,4,5,6,7,8]
n = np.array(n)
g = np.zeros((len(n),),dtype=np.int32)
print(g)
a = [0,5,7]
for i in a:
    print(np.array(i == np.arange(0, len(n) )))

In [86]:
def inner_pro(x1,x2,y1,y2):
    return (x1*y1)+(x2*y2)
x = [1.4, 1.6]
A1 = [1.5, 2.0, 1.6, 1.2, 1.5]
A2 = [1.7, 1.9, 1.8, 1.5, 1.0]
ans = []
for i in range(5):
    ans.append(inner_pro(x[0],x[1],A1[i],A2[i]) / (inner_pro(x[0],x[1],x[0],x[1]) * inner_pro(A1[i],A2[i],A1[i],A2[i])))
ans

[0.2074653076684687,
 0.16978126126545184,
 0.19530057979859625,
 0.2446219152457011,
 0.2518720217835262]

In [68]:
f = frozenset([1,2,3,4,5])
for i in f:
    print(i)

5
